In [0]:
#Mounting Goole drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# clone darknet repo
%cd /content/
!git clone https://github.com/ansarisam/darknet

!/usr/local/cuda/bin/nvcc --version

In [0]:
# change makefile to have GPU and OPENCV enabled
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [0]:
# make darknet (build)
%cd /content/darknet/
!make

In [0]:
#Content from drive
%cd '/content/drive/My Drive'
%cp -av Potholes/ /content/Colab_Potholes

In [0]:
#Content from drive
%cd '/content/drive/My Drive'
%cp -av files/ /content/

In [0]:
### XML to .txt ###
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_txt(here):
    #cls=open('/content/drive/My Drive/DAEN/class.data')
    xml_list = ''
    tree = ET.parse(here)
    root = tree.getroot()
    
    for member in root.findall('object'):
        bbx = member.find('bndbox')
        xmin = float(bbx.find('xmin').text)
        ymin = float(bbx.find('ymin').text)
        xmax = float(bbx.find('xmax').text)
        ymax = float(bbx.find('ymax').text)
        label = member.find('name').text
        
        dw=float(1/(int(root.find('size')[0].text)))
        dh=float(1/(int(root.find('size')[1].text)))

        x=float((((xmin+xmax)/2)-1)*dw)
        y=float((((ymin+ymax)/2)-1)*dh)

        h=float((ymax-ymin)*dh)
        w=float((xmax-xmin)*dw)
        
        filename=root.find('filename').text

        xml_list=xml_list + str(0)+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)+'\n'
        
    #column_name = ['class', 'x_center', 'y_center', 'width', 'height']
    f=open('/content/Colab_Potholes/labels/'+filename.rsplit('.',1)[0]+".txt",'w')
    f.write(xml_list)
    f.close()
    
def main():
    file_path="/content/Colab_Potholes/Annotations"
    read_files=glob.glob(os.path.join(file_path,"*.xml"))
    length=len(os.listdir(file_path))
    for i in read_files:
      xml_df = xml_to_txt(i)
      
main()

In [0]:
### Create train.txt ###
import pandas as pd
df=pd.read_csv('/content/Colab_Potholes/Main/Pothole_train.txt',header=None)
df=df[0].str.split(" ",n=1,expand=True)
df=df.drop(columns=1,axis=1)
df['value']='/content/Colab_Potholes/images/'
df['final']=df['value']+df[0]
df

a = open('/content/Colab_Potholes/data/train.txt', 'w')
for files in df['final']:
  a.write(files+'\n')
a.close()

In [0]:
### Create test.txt ###
import pandas as pd
df=pd.read_csv('/content/Colab_Potholes/Main/Pothole_val.txt',header=None)
df=df[0].str.split(" ",n=1,expand=True)
df=df.drop(columns=1,axis=1)
df['value']='/content/Colab_Potholes/images/'
df['final']=df['value']+df[0]
df

b = open('/content/Colab_Potholes/data/test.txt', 'w')
for files in df['final']:
  b.write(files+'\n')
b.close()

In [0]:
### Pretrained weights ###
%%shell
mkdir /content/pretrained
cd /content/pretrained
wget https://pjreddie.com/media/files/darknet53.conv.74

In [0]:
### dos2unix ###
%%shell
sudo apt install dos2unix
dos2unix /content/files/input.cfg

In [0]:
### Model training ###
%%shell
mkdir -p /content/backup
cd /content/darknet
./darknet detector train /content/files/input.cfg /content/files/yolov3-voc.cfg /content/yolov3-voc_5200.weights

In [0]:
### Model testing ###
%%shell
cd /content/darknet
./darknet detector test /content/files/input.cfg /content/files/yolov3-voc-testing.cfg /content/files/yolov3-voc_final.weights

In [0]:
### AlexeyAB (file structure) ###
#!cp /content/Colab_Potholes/labels/*.txt /content/Colab_Potholes/images

In [0]:
### map Value using AlexeyAB repo ###
%%shell
mkdir -p /content/backup
cd /content/darknet
./darknet detector map /content/files/input.cfg /content/files/yolov3-voc.cfg /content/yolov3-voc_5200.weights

In [0]:
### Test image to JSON ###
%%shell
mkdir -p /content/output
cd /content/darknet/
python /content/files/testimages_json.py

In [0]:
### BBOX on Test Images ###
import cv2,os,json
from google.colab.patches import cv2_imshow

color=(0,0,255)
thickness=1
for files in os.listdir('/content/output/'):
  with open("/content/output/"+files, 'r') as f:
    test_json = json.load(f)
  if len(test_json) == 0:
    continue
  else:
    print(test_json[0]['image'])
    path=test_json[0]['image']
    img=cv2.imread(path)
    for i in range(0,len(test_json)):
      a=test_json[i]['predictions']['bbox']
      start=(int(a[0]),int(a[1]))
      end=(int(a[2]),int(a[3]))
      x=int(a[0])
      y=int(a[1])-1
      predictedimg=cv2.rectangle(img,start,end,color,thickness)
      predictedimg=cv2.putText(predictedimg,'Pothole:'+test_json[i]['predictions']['score'],(x,y),cv2.FONT_HERSHEY_PLAIN,1,color,1,cv2.LINE_AA)
    cv2_imshow(predictedimg)

In [0]:
### Detection in a video ###
%%shell
python /content/files/yolo_video.py --input /content/Pothole_Hell.mp4 --output /content/drive/'My Drive'/11_5200.avi --yolo /content/files

[INFO] loading YOLO from disk...
[INFO] 2284 total frames in video
[INFO] single frame took 2.6111 seconds
[INFO] estimated total time to finish: 5963.7240
[INFO] cleaning up...


In [0]:
### Plotting loss ###
!grep "avg" /content/logs_6000.log > /content/logs.txt
import sys
import matplotlib.pyplot as plt

lines = []
for line in open('/content/logs.txt'):
    if "avg" in line:
        lines.append(line)

iterations = []
avg_loss = []

for i in range(len(lines)):
    lineParts = lines[i].split(',')
    iterations.append(int(lineParts[0].split(':')[0]))
    avg_loss.append(float(lineParts[1].split()[0]))

fig = plt.figure()
for i in range(0, len(lines)):
    plt.plot(iterations[i:i+1], avg_loss[i:i+1],'r.-',markersize=2)
    #print(iterations[i:i+1], avg_loss[i:i+1])

plt.xlabel('Batch Number')
plt.ylabel('Avg Loss')
fig.savefig("/content/"+'training_loss_plot.png', dpi=1000)

In [0]:
### Location using exit lib ###
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif(filename):
    exif = Image.open(filename)._getexif()

    if exif is not None:
        for key, value in exif.items():
            name = TAGS.get(key, key)
            exif[name] = exif.pop(key)

        if 'GPSInfo' in exif:
            for key in exif['GPSInfo'].keys():
                name = GPSTAGS.get(key,key)
                exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)

    return exif

exif = get_exif('/content/WIN_20200426_18_26_02_Pro.jpg')
print(exif['GPSInfo'])

In [1]:
### Clone exiftool ###
%cd /content/
!git clone https://github.com/exiftool/exiftool

/content
Cloning into 'exiftool'...
^C


In [0]:
### Location using exiftool ###
%cd /content/exiftool/
!./exiftool -gpsposition /content/IMG_4504.MOV

/content/exiftool
GPS Position                    : 38 deg 49' 52.32" N, 77 deg 18' 42.12" W
